In [6]:
# making the dataset
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [7]:
# using train test split to split the database
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X, y,test_size=0.35, random_state=42)

In [11]:
# defining the model
from sklearn.tree import DecisionTreeClassifier

d_tree = DecisionTreeClassifier(max_depth=2)

In [12]:
# using gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

gsc = GridSearchCV(
        estimator= d_tree,
        param_grid={
             'max_leaf_nodes': [2,4,6,8,10]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1
        
    )

In [16]:
#now finding the best params
grid_result = gsc.fit(X_test,y_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished


In [19]:
best_params = grid_result.best_params_
print(best_params)

{'max_leaf_nodes': 4}


In [20]:
best_tree = DecisionTreeClassifier(max_depth=2,max_leaf_nodes=4)

In [21]:
best_tree.fit(X_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [22]:
# testing the accuracy
from sklearn.metrics import accuracy_score

y_pred = gsc.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8602857142857143


### actual way

In [ ]:
params = {'max_leaf_nodes:'list(range(2,100)), ""}